In [5]:
class Tolerant_retrieval:
    
    
    def start(self, cells):
        english_document = []
        for row in cells.iter_rows():
            index = 0;
            for k in row:
                if index == 0:
                    english_document.append(k.internal_value.lower())
                    break
        tokens = self.generate_tokens(english_document)
        pruned_tokens = self.remove_stop_words(tokens)
        
        return pruned_tokens

    def generate_tokens(self, english_document):
        final_tokens = []
        for i in range(len(english_document)):

            nltk_tokens = nltk.word_tokenize(english_document[i])
            #print(nltk_tokens)
            final_tokens.append(nltk_tokens)
        #rint(final_tokens)
        return final_tokens
        
    def remove_stop_words(self, tokens):
    
        eng_stop_words = stopwords.words('english')
        little_pruned_tokens = []
        pruned_tokens = []

        for i in range(len(tokens)):
            temp = []
            for j in range(len(tokens[i])):
                if tokens[i][j] not in eng_stop_words:
                    temp.append(tokens[i][j])    
            little_pruned_tokens.append(temp)


        for i in range(len(little_pruned_tokens)):
            temp = []
            for j in range(len(little_pruned_tokens[i])):        
                if little_pruned_tokens[i][j].isalpha():
                    temp.append(little_pruned_tokens[i][j])

            pruned_tokens.append(temp)

        return pruned_tokens

    def stemming(self, pruned_tokens):
    
        ps = PorterStemmer()
        stemmed_words = []

        for i in range(len(pruned_tokens)):
            temp = []
            for j in range(len(pruned_tokens[i])):        
                temp.append(ps.stem(pruned_tokens[i][j]))

            stemmed_words.append(temp)

        return stemmed_words
    
    def dictionary_of_words(self, stemmed_words):
    
        dictionary = {}

        for i in range(len(stemmed_words)):
            for j in range(len(stemmed_words[i])):        
                #count = frequency.get(stemmed_words[i][j],0)
                dictionary[stemmed_words[i][j]] = 1
        
        sorted_dict = self.sortedDictkeys(dictionary)
        
        return sorted_dict
    
    def sortedDictkeys(self, adict):
        sort_dict = {}

        for (key, value) in sorted(adict.items()):
            sort_dict[key] = value

        return sort_dict
    
    def bigrams_postings(self, dictionary):

        bigram = defaultdict(list)

        for key, date in dictionary.items():
            for i in range(len(key)-1):
                bivalue = key[i:i+2]
                bigram[bivalue].append(key)

        return bigram
    
    def jaccard_similarity(self, dict_word, query_word):
        list1 = []
        list2 = []

        for i in range(len(dict_word)-1):
            list1.append(dict_word[i:i+2]) 

        for i in range(len(query_word)-1):
            list2.append(query_word[i:i+2]) 

        intersection = len(set(list1).intersection(set(list2)))
        union = (len(dict_word)-1) + (len(query_word)) - intersection

        return float(intersection/union)
    
    def suggested_corrected_words(self, bigram_dict, query_word):
    
        selected_word = set()
        for i in range(len(query_word)-1):
            key = query_word[i:i+2]

            for word in bigram_dict[key]:
                similarity = self.jaccard_similarity(word, query_word)
                #x = float(nltk.jaccard_distance(set(word), set(query_word)))

                if similarity > 0.30:
                #if 1-x > 0.70:
                    selected_word.add(word)
            
            min_len = 80
            correct_word = ""
            for word in selected_word:
                #print(word)
                dist = self.edit_distance(word, query_word)
                if min_len >= dist:
                    min_len = dist
                    correct_word = word

        return correct_word
    
    def edit_distance(self, word, query_word):
    
        if word == query_word: return 0
        elif len(word) == 0: return len(query_word)
        elif len(query_word) == 0: return len(word)
        dp0 = [None] * (len(query_word) + 1)
        dp1 = [None] * (len(query_word) + 1)
        for i in range(len(dp0)):
            dp0[i] = i
        for i in range(len(word)):
            dp1[0] = i + 1
            for j in range(len(query_word)):
                cost = 0 if word[i] == query_word[j] else 1
                dp1[j + 1] = min(dp1[j] + 1, dp0[j + 1] + 1, dp0[j] + cost)
            for j in range(len(dp0)):
                dp0[j] = dp1[j]
                
        return dp1[len(query_word)]
    
    def boolean_retrieval(self, bigram_dict, queries, inverted_index):
        
        ps = PorterStemmer()
        bool_result = []
        correct_queries = []
        
        for i in range(len(queries)):
            connecting_words = []
            cnt = 1
            different_words = []

            for word in queries[i]:
                if word.lower() != "and" and word.lower() != "or" and word.lower() != "not":
                    different_words.append(word.lower())
                else:
                    connecting_words.append(word.lower())
#             print(connecting_words)
#             print(different_words)
#             break
            
            for j in range(len(different_words)):
                different_words[j] = self.suggested_corrected_words(bigram_dict, different_words[j])
                #print(different_words[j])
                
            result = inverted_index[ps.stem(different_words[0])]
            result_query = different_words[0]
            
            if len(different_words) == 1 and len(connecting_words) == 1:
                result = self.not_merging_list(result)
                result_query = 'not' + ' ' + result_query
                
            else:
                m = 1
                k = 0
                while k != len(connecting_words):
                    flag = 1
                    #print(k, m)
                    list1 = inverted_index[ps.stem(different_words[m])]
                    #print(list1)
                    if connecting_words[k] == 'and':
                        if k+1 < len(connecting_words) and connecting_words[k+1] == 'not':
                            list2 = self.not_merging_list(list1)
                            result = self.and_merging_list(result, list2)
                            flag = 0
                            op = 'and'
                            k = k+1
                        else:
                            result = self.and_merging_list(result, list1)
                    elif connecting_words[k] == 'or':
                        if k+1 < len(connecting_words) and connecting_words[k+1] == 'not':
                            list2 = self.not_merging_list(list1)
                            result = self.or_merging_list(result, list2)
                            flag = 0
                            op = 'or'
                            k = k+1
                        else:
                            result = self.or_merging_list(result, list1)
                    else:
                        result = not_merging_list(result, list1)

                    if flag == 1:
                        result_query = result_query + " " + connecting_words[k] + " " + different_words[m]
                    else:
                        result_query = result_query + " " + op + " " + connecting_words[k] + " " + different_words[m]
                    m = m+1 
                    k = k+1

            bool_result.append(result)
            correct_queries.append(result_query)

        return bool_result, correct_queries

    def not_merging_list(self, list2):
        result = []
        k = 0
        for i in range(2276):
            #print(i, list2[k])
            if k < len(list2) and i == list2[k]:
                k = k+1
            else:
                result.append(i)
                
        return result

    
    def and_merging_list(self, list1, list2):
        i = 0
        j = 0
        result = []

        while i != len(list1) and j != len(list2):
            if list1[i] == list2[j]:
                result.append(list1[i])
                i = i+1
                j = j+1
            elif list1[i] > list2[j]:
                j = j+1
            else:
                i = i+1

        return result
    
    def or_merging_list(self, list1, list2):
        i = 0
        j = 0
        result = []

        while i != len(list1) and j != len(list2):
            if list1[i] == list2[j]:
                result.append(list1[i])
                i = i+1
                j = j+1
            elif list1[i] > list2[j]:
                result.append(list2[j])
                j = j+1
            else:
                result.append(list1[i])
                i = i+1

        while i != len(list1):
            result.append(list1[i])
            i = i+1

        while j != len(list2):
            result.append(list2[j])
            j = j+1

        return result
    
    def inverted_index_fun(self, documents):

        inverted_index = defaultdict(set)

        for i in range(len(documents)):
            for j in range(len(documents[i])):

                inverted_index[documents[i][j]].add(i)

        #sorting the dictionary by values:
        for keys, value in inverted_index.items():
            tmp = sorted(value)
            inverted_index[keys] = tmp

        return inverted_index
    
    def get_documents_postings(self, word):
        return self.inverted_index_fun(word)

In [6]:
import openpyxl as px
import codecs
import string
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import defaultdict
import sys

[nltk_data] Downloading package punkt to /home/dell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/dell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def main():
    W = px.load_workbook('Dand_Prakriya.xlsx')
    p = W.get_sheet_by_name(name = 'Sheet')
    english_document = []
    
    ret = Tolerant_retrieval()
    
    pruned_tokens = ret.start(p)
    stemmed_words = ret.stemming(pruned_tokens)
    inverted_index = ret.inverted_index_fun(stemmed_words)
    #print(inverted_index["government"])
    #print(inverted_index["state"])
    dictionary = ret.dictionary_of_words(pruned_tokens)
    #print(dictionary)
    #correcting queries:
    bigram_dict = ret.bigrams_postings(dictionary)
    
    f=codecs.open("query_eng.txt",encoding='utf-8')
    search = [x.strip() for x in f.readlines()]
    queries = ret.generate_tokens(search)
    
    boolean_docs, correct_queries = ret.boolean_retrieval(bigram_dict, queries, inverted_index)
    
    return boolean_docs, correct_queries

In [8]:
if __name__ == '__main__':
    boolean_docs, correct_queries = main()

    std_out = sys.stdout
    file = open('OUT_ENGLISH.txt', 'w')
    sys.stdout = file

    for i in range(len(correct_queries)):
        print(correct_queries[i], "=>", boolean_docs[i])

    sys.stdout = std_out
    file.close()

/home/dell/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  This is separate from the ipykernel package so we can avoid doing imports until
